# 指南：使用阶跃星辰自动上下文切换模型

本 `ipynb` 文件旨在解释和演示阶跃星辰（StepFun）API 提供的 **自动上下文切换模型** (`step-1-auto` 和 `step-1v-auto`)。该功能可以根据输入内容的长度，智能地选择最合适的底层模型进行推理，从而帮助开发者在控制成本和处理长文本之间取得平衡。

## 1. 什么是自动上下文切换模型？

阶跃星辰平台提供了两个特殊的模型名称（“虚拟模型”）：

- `step-1-auto`
- `step-1v-auto` (用于视觉模型)

这两个模型 **不是全新的模型**，而是作为一种路由机制存在的。当你使用它们时，平台会根据你输入的文本长度（Token 数量）加上你设定的 `max_tokens`，自动选择一个最经济、最合适的底层模型来处理你的请求。

**`step-1-auto` 的切换逻辑：**

| 总 Token 数 (输入 + max_tokens) | 实际调用的底层模型 |
| :--- | :--- |
| <= 8,192 | `step-1-8k` |
| > 8,192 and <= 32,768 | `step-1-32k` |
| > 32,768 | `step-1-256k` |

**`step-1v-auto` 的切换逻辑：**

| 总 Token 数 (输入 + max_tokens) | 实际调用的底层模型 |
| :--- | :--- |
| <= 8,192 | `step-1v-8k` |
| > 8,192 | `step-1v-32k` |

## 2. 如何开启

开启此功能非常简单，只需在调用 API 时，将 `model` 参数设置为 `step-1-auto` 或 `step-1v-auto` 即可。

In [ ]:
import os
from openai import OpenAI

# 注意：请将 'YourAPIKey' 替换为您自己的阶跃星辰 API Key
STEPFUN_API_KEY = os.getenv("STEPFUN_API_KEY")

client = OpenAI(
    base_url="https://api.stepfun.com/v1",
    api_key=STEPFUN_API_KEY
)

# 一个简短的输入
short_messages = [
    {"role": "user", "content": "你好！"}
]

# 一个非常长的输入
long_content = """

以下道德经全文，将其翻译成现代汉语：
道可道，非常道。名可名，非常名。无名天地之始；有名万物之母。故常无，欲以观其妙；常有，欲以观其徼。此两者，同出而异名，同谓之玄。玄之又玄，众妙之门。天下皆知美之为美，斯恶已。皆知善之为善，斯不善已。故有无相生，难易相成，长短相形，高下相倾，音声相和，前后相随。是以圣人处无为之事，行不言之教；万物作焉而不辞，生而不有，为而不恃，功成而弗居。夫唯弗居，是以不去。不尚贤，使民不争；不贵难得之货，使民不为盗；不见可欲，使民心不乱。是以圣人之治，虚其心，实其腹，弱其志，强其骨。常使民无知无欲，使夫智者不敢为也。为无为，则无不治。道冲而用之或不盈，渊兮似万物之宗。挫其锐，解其纷，和其光，同其尘。湛兮似或存，吾不知谁之子，象帝之先。天地不仁，以万物为刍狗；圣人不仁，以百姓为刍狗。天地之间，其犹橐龠乎？虚而不屈，动而愈出。多言数穷，不如守中。谷神不死，是谓玄牝。玄牝之门，是谓天地根。绵绵若存，用之不勤。天长地久。天地所以能长且久者，以其不自生，故能长生。是以圣人后其身而身先；外其身而身存。非以其无私邪？故能成其私。上善若水。水善利万物而不争，处众人之所恶，故几于道。居善地，心善渊，与善仁，言善信，政善治，事善能，动善时。夫唯不争，故无尤。持而盈之，不如其已；揣而锐之，不可长保。金玉满堂，莫之能守；富贵而骄，自遗其咎。功遂身退，天之道也。载营魄抱一，能无离乎？专气致柔，能如婴儿乎？涤除玄览，能无疵乎？爱民治国，能无为乎？天门开阖，能为雌乎？明白四达，能无知乎？生之、畜之，生而不有，为而不恃，长而不宰，是谓玄德。三十辐共一毂，当其无，有车之用。埏埴以为器，当其无，有器之用。凿户牖以为室，当其无，有室之用。故有之以为利，无之以为用。五色令人目盲；五音令人耳聋；五味令人口爽；驰骋畋猎，令人心发狂；难得之货，令人行妨。是以圣人为腹不为目，故去彼取此。宠辱若惊，贵大患若身。何谓宠辱若惊？宠为下，得之若惊，失之若惊，是谓宠辱若惊。何谓贵大患若身？吾所以有大患者，为吾有身，及吾无身，吾有何患？故贵以身为天下，若可寄天下；爱以身为天下，若可托天下。视之不见，名曰夷；听之不闻，名曰希；搏之不得，名曰微。此三者不可致诘，故混而为一。其上不皦，其下不昧，绳绳兮不可名，复归于无物。是谓无状之状，无物之象，是谓惚恍。迎之不见其首，随之不见其后。执古之道，以御今之有。能知古始，是谓道纪。古之善为士者，微妙玄通，深不可识。夫唯不可识，故强为之容：豫兮若冬涉川；犹兮若畏四邻；俨兮其若客；涣兮其若凌释；敦兮其若朴；旷兮其若谷；混兮其若浊；澹兮其若海；飂兮若无止。孰能浊以静之徐清？孰能安以久动之徐生？保此道者，不欲盈。夫唯不盈，故能蔽而新成。致虚极，守静笃。万物并作，吾以观复。夫物芸芸，各复归其根。归根曰静，静曰复命。复命曰常，知常曰明。不知常，妄作凶。知常容，容乃公，公乃全，全乃天，天乃道，道乃久，没身不殆。太上，不知有之；其次，亲而誉之；其次，畏之；其次，侮之。信不足焉，有不信焉。悠兮其贵言。功成事遂，百姓皆谓：我自然。大道废，有仁义；智慧出，有大伪；六亲不和，有孝慈；国家昏乱，有忠臣。绝圣弃智，民利百倍；绝仁弃义，民复孝慈；绝巧弃利，盗贼无有。此三者以为文，不足。故令有所属：见素抱朴，少思寡欲，绝学无忧。唯之与阿，相去几何？美之与恶，相去若何？人之所畏，不可不畏。荒兮其未央哉！众人熙熙，如享太牢，如春登台。我独泊兮其未兆，如婴儿之未孩；儽儽兮若无所归。众人皆有余，而我独若遗。我愚人之心也哉！沌沌兮！俗人昭昭，我独昏昏。俗人察察，我独闷闷。澹兮其若海，飂兮若无止。众人皆有以，而我独顽且鄙。我独异于人，而贵食母。孔德之容，惟道是从。道之为物，惟恍惟惚。惚兮恍兮，其中有象；恍兮惚兮，其中有物。窈兮冥兮，其中有精；其精甚真，其中有信。自今及古，其名不去，以阅众甫。吾何以知众甫之状哉？以此。曲则全，枉则直，洼则盈，敝则新，少则得，多则惑。是以圣人抱一为天下式。不自见，故明；不自是，故彰；不自伐，故有功；不自矜，故长。夫唯不争，故天下莫能与之争。古之所谓「曲则全」者，岂虚言哉！诚全而归之。希言自然。故飘风不终朝，骤雨不终日。孰为此者？天地。天地尚不能久，而况于人乎？故从事于道者，同于道；德者，同于德；失者，同于失。同于道者，道亦乐得之；同于德者，德亦乐得之；同于失者，失亦乐得之。信不足焉，有不信焉。跂者不立；跨者不行；自见者不明；自是者不彰；自伐者无功；自矜者不长。其在道也，曰：余食赘形。物或恶之，故有道者不处。有物混成，先天地生。寂兮寥兮，独立而不改，周行而不殆，可以为天地母。吾不知其名，强字之曰道，强为之名曰大。大曰逝，逝曰远，远曰反。故道大，天大，地大，人亦大。域中有四大，而人居其一焉。人法地，地法天，天法道，道法自然。重为轻根，静为躁君。是以君子终日行不离辎重，虽有荣观，燕处超然。奈何万乘之主，而以身轻天下？轻则失根，躁则失君。善行无辙迹；善言无瑕谪；善数不用筹策；善闭无关楗而不可开；善结无绳约而不可解。是以圣人常善救人，故无弃人；常善救物，故无弃物。是谓袭明。故善人者，不善人之师；不善人者，善人之资。不贵其师，不爱其资，虽智大迷，是谓要妙。知其雄，守其雌，为天下溪。为天下溪，常德不离，复归于婴儿。知其白，守其黑，为天下式。为天下式，常德不忒，复归于无极。知其荣，守其辱，为天下谷。为天下谷，常德乃足，复归于朴。朴散则为器，圣人用之，则为官长，故大制不割。将欲取天下而为之，吾见其不得已。天下神器，不可为也，不可执也。为者败之，执者失之。故物或行或随，或嘘或吹，或强或羸，或载或隳。是以圣人去甚，去奢，去泰。以道佐人主者，不以兵强天下。其事好还。师之所处，荆棘生焉。大军之后，必有凶年。善有果而已，不以取强。果而勿矜，果而勿伐，果而勿骄，果而不得已，果而勿强。物壮则老，是谓不道，不道早已。夫兵者，不祥之器，物或恶之，故有道者不处。君子居则贵左，用兵则贵右。兵者不祥之器，非君子之器，不得已而用之，恬淡为上。胜而不美，而美之者，是乐杀人。夫乐杀人者，则不可得志于天下矣。吉事尚左，凶事尚右。偏将军居左，上将军居右，言以丧礼处之。杀人之众，以悲哀泣之，战胜以丧礼处之。道常无名。朴虽小，天下莫能臣。侯王若能守之，万物将自宾。天地相合，以降甘露，民莫之令而自均。始制有名，名亦既有，夫亦将知止，知止可以不殆。譬道之在天下，犹川谷之于江海。知人者智，自知者明。胜人者有力，自胜者强。知足者富。强行者有志。不失其所者久。死而不亡者寿。大道泛兮，其可左右。万物恃之以生而不辞，功成而不有。衣养万物而不为主，可名于小；万物归焉而不为主，可名为大。以其终不自为大，故能成其大。执大象，天下往。往而不害，安平泰。乐与饵，过客止。道之出口，淡乎其无味，视之不足见，听之不足闻，用之不足既。将欲歙之，必固张之；将欲弱之，必固强之；将欲废之，必固兴之；将欲取之，必固与之。是谓微明。柔弱胜刚强。鱼不可脱于渊，国之利器不可以示人。道常无为而无不为。侯王若能守之，万物将自化。化而欲作，吾将镇之以无名之朴。镇之以无名之朴，夫将不欲。不欲以静，天下将自正。上德不德，是以有德；下德不失德，是以无德。上德无为而无以为；下德无为而有以为。上仁为之而无以为；上义为之而有以为。上礼为之而莫之应，则攘臂而扔之。故失道而后德，失德而后仁，失仁而后义，失义而后礼。夫礼者，忠信之薄，而乱之首。前识者，道之华，而愚之始。是以大丈夫处其厚，不居其薄；处其实，不居其华。故去彼取此。昔之得一者：天得一以清；地得一以宁；神得一以灵；谷得一以盈；万物得一以生；侯王得一以为天下贞。其致之也，谓天无以清，将恐裂；地无以宁，将恐废；神无以灵，将恐歇；谷无以盈，将恐竭；万物无以生，将恐灭；侯王无以贞，将恐蹶。故贵以贱为本，高以下为基。是以侯王自称孤、寡、不谷。此非以贱为本邪？非乎？故致誉无誉。是故不欲琭琭如玉，珞珞如石。反者道之动；弱者道之用。天下万物生于有，有生于无。上士闻道，勤而行之；中士闻道，若存若亡；下士闻道，大笑之。不笑不足以为道。故建言有之：明道若昧；进道若退；夷道若颣；上德若谷；广德若不足；建德若偷；质真若渝；大白若辱；大方无隅；大器晚成；大音希声；大象无形；道隐无名。夫唯道，善贷且成。道生一，一生二，二生三，三生万物。万物负阴而抱阳，冲气以为和。人之所恶，唯孤、寡、不谷，而王公以为称。故物或损之而益，或益之而损。人之所教，我亦教之。强梁者不得其死，吾将以为教父。天下之至柔，驰骋天下之至坚。无有入无间，吾是以知无为之有益。不言之教，无为之益，天下希及之。名与身孰亲？身与货孰多？得与亡孰病？甚爱必大费；多藏必厚亡。故知足不辱，知止不殆，可以长久。大成若缺，其用不弊。大盈若冲，其用不穷。大直若屈，大巧若拙，大辩若讷。静胜躁，寒胜热。清静为天下正。天下有道，却走马以粪。天下无道，戎马生于郊。祸莫大于不知足；咎莫大于欲得。故知足之足，常足矣。不出户，知天下；不窥牖，见天道。其出弥远，其知弥少。是以圣人不行而知，不见而明，不为而成。为学日益，为道日损。损之又损，以至于无为。无为而无不为。取天下常以无事，及其有事，不足以取天下。圣人无常心，以百姓心为心。善者，吾善之；不善者，吾亦善之；德善。信者，吾信之；不信者，吾亦信之；德信。圣人在天下，歙歙焉，为天下浑其心，百姓皆注其耳目，圣人皆孩之。出生入死。生之徒，十有三；死之徒，十有三；人之生，动之于死地，亦十有三。夫何故？以其生生之厚。盖闻善摄生者，陆行不遇兕虎，入军不被甲兵；兕无所投其角，虎无所用其爪，兵无所容其刃。夫何故？以其无死地。道生之，德畜之，物形之，势成之。是以万物莫不尊道而贵德。道之尊，德之贵，夫莫之命而常自然。故道生之，德畜之；长之育之；亭之毒之；养之覆之。生而不有，为而不恃，长而不宰，是谓玄德。天下有始，以为天下母。既得其母，以知其子；既知其子，复守其母，没身不殆。塞其兑，闭其门，终身不勤。开其兑，济其事，终身不救。见小曰明，守柔曰强。用其光，复归其明，无遗身殃；是为袭常。使我介然有知，行于大道，唯施是畏。大道甚夷，而人好径。朝甚除，田甚芜，仓甚虚；服文彩，带利剑，厌饮食，财货有余；是为盗夸。非道也哉！善建者不拔，善抱者不脱，子孙以祭祀不辍。修之于身，其德乃真；修之于家，其德乃余；修之于乡，其德乃长；修之于邦，其德乃丰；修之于天下，其德乃普。故以身观身，以家观家，以乡观乡，以邦观邦，以天下观天下。吾何以知天下然哉？以此。含德之厚，比于赤子。毒虫不螫，猛兽不据，攫鸟不搏。骨弱筋柔而握固。未知牝牡之合而朘作，精之至也。终日号而不嗄，和之至也。知和曰常，知常曰明。益生曰祥。心使气曰强。物壮则老，谓之不道，不道早已。知者不言，言者不知。挫其锐，解其纷，和其光，同其尘，是谓「玄同」。故不可得而亲，不可得而疏；不可得而利，不可得而害；不可得而贵，不可得而贱。故为天下贵。以正治国，以奇用兵，以无事取天下。吾何以知其然哉？以此：天下多忌讳，而民弥贫；人多利器，国家滋昏；人多伎巧，奇物滋起；法令滋彰，盗贼多有。故圣人云：我无为，而民自化；我好静，而民自正；我无事，而民自富；我无欲，而民自朴。其政闷闷，其民淳淳；其政察察，其民缺缺。祸兮福之所倚，福兮祸之所伏。孰知其极？其无正也。正复为奇，善复为妖。人之迷，其日固久。是以圣人方而不割，廉而不刿，直而不肆，光而不耀。治人事天，莫若啬。夫唯啬，是谓早服；早服谓之重积德；重积德则无不克；无不克则莫知其极；莫知其极，可以有国；有国之母，可以长久；是谓深根固柢，长生久视之道。治大国，若烹小鲜。以道莅天下，其鬼不神；非其鬼不神，其神不伤人；非其神不伤人，圣人亦不伤人。夫两不相伤，故德交归焉。大邦者下流，天下之牝，天下之交也。牝常以静胜牡，以静为下。故大邦以下小邦，则取小邦；小邦以下大邦，则取大邦。故或下以取，或下而取。大邦不过欲兼畜人，小邦不过欲入事人。夫两者各得所欲，大者宜为下。道者万物之奥。善人之宝，不善人之所保。美言可以市尊，美行可以加人。人之不善，何弃之有？故立天子，置三公，虽有拱璧以先驷马，不如坐进此道。古之所以贵此道者何？不曰：求以得，有罪以免邪？故为天下贵。为无为，事无事，味无味。大小多少，报怨以德。图难于其易，为大于其细；天下难事，必作于易；天下大事，必作于细。是以圣人终不为大，故能成其大。夫轻诺必寡信，多易必多难。是以圣人犹难之，故终无难矣。其安易持，其未兆易谋。其脆易泮，其微易散。为之于未有，治之于未乱。合抱之木，生于毫末；九层之台，起于累土；千里之行，始于足下。民之从事，常于几成而败之。慎终如始，则无败事。古之善为道者，非以明民，将以愚之。民之难治，以其智多。故以智治国，国之贼；不以智治国，国之福。知此两者亦稽式。常知稽式，是谓「玄德」。玄德深矣，远矣，与物反矣，然后乃至大顺。江海所以能为百谷王者，以其善下之，故能为百谷王。是以圣人欲上民，必以言下之；欲先民，必以身后之。是以圣人处上而民不重，处前而民不害。是以天下乐推而不厌。以其不争，故天下莫能与之争。天下皆谓我道大，似不肖。夫唯大，故似不肖。若肖，久矣其细也夫！我有三宝，持而保之。一曰慈，二曰俭，三曰不敢为天下先。慈故能勇；俭故能广；不敢为天下先，故能成器长。今舍慈且勇；舍俭且广；舍后且先；死矣！夫慈以战则胜，以守则固。天将救之，以慈卫之。善为士者，不武；善战者，不怒；善胜敌者，不与；善用人者，为之下。是谓不争之德，是谓用人之力，是谓配天古之极。用兵有言：「吾不敢为主，而为客；不敢进寸，而退尺。」是谓行无行；攘无臂；扔无敌；执无兵。祸莫大于轻敌，轻敌几丧吾宝。故抗兵相若，哀者胜矣。吾言甚易知，甚易行。天下莫能知，莫能行。言有宗，事有君。夫唯无知，是以不我知。知我者希，则我者贵。是以圣人被褐而怀玉。知不知，尚矣；不知知，病也。圣人不病，以其病病。夫唯病病，是以不病。民不畏威，则大威至。无狎其所居，无厌其所生。夫唯不厌，是以不厌。是以圣人自知不自见；自爱不自贵。故去彼取此。勇于敢则杀，勇于不敢则活。此两者，或利或害。天之所恶，孰知其故？天之道，不争而善胜，不言而善应，不召而自来，繟然而善谋。天网恢恢，疏而不失。民不畏死，奈何以死惧之？若使民常畏死，而为奇者，吾得执而杀之，孰敢？常有司杀者杀。夫代司杀者杀，是谓代大匠斲，夫代大匠斲者，希有不伤其手矣。民之饥，以其上食税之多，是以饥。民之难治，以其上之有为，是以难治。民之轻死，以其上求生之厚，是以轻死。夫唯无以生为者，是贤于贵生。人之生也柔弱，其死也坚强。草木之生也柔脆，其死也枯槁。故坚强者死之徒，柔弱者生之徒。是以兵强则灭，木强则折。强大处下，柔弱处上。天之道，其犹张弓欤？高者抑之，下者举之；有余者损之，不足者补之。天之道，损有余而补不足。人之道，则不然，损不足以奉有余。孰能有余以奉天下，唯有道者。是以圣人为而不恃，功成而不处，其不欲见贤。天下莫柔弱于水，而攻坚强者莫之能胜，以其无以易之。弱之胜强，柔之胜刚，天下莫不知，莫能行。是以圣人云：「受国之垢，是谓社稷主；受国不祥，是为天下王。」正言若反。和大怨，必有余怨；安可以为善？是以圣人执左契，而不责于人。有德司契，无德司彻。天道无亲，常与善人。小国寡民。使有什伯之器而不用；使民重死而不远徙。虽有舟舆，无所乘之；虽有甲兵，无所陈之。使民复结绳而用之。甘其食，美其服，安其居，乐其俗。邻国相望，鸡犬之声相闻，民至老死，不相往来。信言不美，美言不信。善者不辩，辩者不善。知者不博，博者不知。圣人不积，既以为人己愈有，既以与人己愈多。天之道，利而不害；圣人之道，为而不争。

"""

long_messages = [
    {"role": "user", "content": long_content}
]

print("--- 正在使用 step-1-auto 模型处理请求 ---\n")

try:
    # 调用短文本，平台会自动选择 step-1-8k
    print("处理短文本:")
    completion_short = client.chat.completions.create(
        model="step-1-auto",
        messages=short_messages,
        max_tokens=50
    )
    print(f"  模型返回: {completion_short.choices[0].message.content}")
    print(f"  实际使用的模型: {completion_short.model}\n") # 返回结果会显示实际使用的模型

    # 调用长文本，平台会自动选择 step-1-32k
    print("处理长文本:")
    completion_long = client.chat.completions.create(
        model="step-1-auto",
        messages=long_messages,
        max_tokens=5000
    )
    print(long_messages)
    print(f"  模型返回: {completion_long.choices[0].message.content}")

    print(f"  模型返回: ... (内容过长，已省略) ...")
    print(f"  实际使用的模型: {completion_long.model}\n")
    print(completion_long)
except Exception as e:
    print(f"API 调用失败: {e}")

--- 正在使用 step-1-auto 模型处理请求 ---

处理短文本:
  模型返回: 你好！今天有什么我可以帮你的吗？如果你有任何问题或需要帮助，请随时告诉我。如果你只是想聊聊天，我也很乐意陪你聊天！请随意。
  实际使用的模型: step-1-8k

处理长文本:
  模型返回: 你好！今天有什么我可以帮你的吗？如果你有任何问题或需要帮助，请随时告诉我。如果你只是想聊聊天，我也很乐意陪你聊天！请随意。
  模型返回: ... (内容过长，已省略) ...
  实际使用的模型: step-1-32k

ChatCompletion(id='6bc1fded08728461b348673bd859ac3a.9c7427b83cfe8c5e49f45b725f772bd0', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='以下是《道德经》全文的现代汉语翻译：\n\n1. **道可道，非常道。名可名，非常名。**\n   - 可以说出的道，不是永恒的道。可以命名的名，不是永恒的名。\n\n2. **无名天地之始；有名万物之母。故常无，欲以观其妙；常有，欲以观其徼。**\n   - 无（道）是天地的本始，有（名）是万物的根源。所以，常从无中观察道的奥妙，常从有中观察道的端倪。\n\n3. **此两者，同出而异名，同谓之玄。玄之又玄，众妙之门。**\n   - 无和有这两者，来源相同而名称相异，它们都称为玄。玄而又玄，是一切奥妙的门径。\n\n4. **天下皆知美之为美，斯恶已。皆知善之为善，斯不善已。**\n   - 天下人都知道美之所以为美，丑的概念也就形成了。都知道善之所以为善，不善的概念也就形成了。\n\n5. **故有无相生，难易相成，长短相形，高下相倾，音声相和，前后相随。**\n   - 所以，有和无互相生成，难和易互相促成，长和短互为显示，高和下互为呈现，音和声彼此应和，前和后相随相依。\n\n6. **是以圣人处无为之事，行不言之教；万物作焉而不辞，生而不有，为而不恃，功成而弗居。夫唯弗居，是以不去。**\n   - 因此，圣人用无为的态度来处理世事，实行不言的教诲；万物兴起而不造作事端，生

## 3. 实现逻辑（伪代码）

平台的后端实现逻辑大致如下：
1.  接收到请求后，首先调用一个内部的 `Toolcall` 来计算输入 `messages` 的 Token 数量。
2.  将计算出的 Token 数与请求中指定的 `max_tokens` 相加，得到总 Token 数。
3.  根据预设的阈值（如 8192, 32768 等），将请求路由到对应的实际模型（`step-1-8k`, `step-1-32k` 等）。

In [ ]:
def pseudo_code_for_auto_model(messages, max_tokens):
    # 1. 计算输入 token 数量
    # 这只是一个示例，实际的 token 计算会更复杂
    prompt_tokens = sum(len(msg['content']) for msg in messages) 
    
    # 2. 计算总 token
    total_tokens = prompt_tokens + max_tokens
    
    # 3. 根据总 token 选择模型
    if total_tokens <= 8192:
        selected_model = "step-1-8k"
    elif total_tokens <= 32768:
        selected_model = "step-1-32k"
    else:
        selected_model = "step-1-256k"
        
    print(f"输入 tokens: {prompt_tokens}, max_tokens: {max_tokens}, 总计: {total_tokens}")
    print(f"自动选择的模型是: {selected_model}")
    
    # 实际会在这里调用相应的模型 API
    # call_actual_model(model=selected_model, messages=messages, max_tokens=max_tokens)

# 模拟调用
pseudo_code_for_auto_model([{"role": "user", "content": "hi"}], 100)
pseudo_code_for_auto_model([{"role": "user", "content": "..." * 5000}], 1000)

## 4. 使用场景

开发者可以根据自己的业务需求，实现更定制化的路由逻辑。例如：

- **处理不确定长度的用户输入**：当无法预知用户会输入多少内容时，使用 `auto` 模型可以避免因超出上下文长度而导致的错误。
- **函数调用（Tool Call）或知识库**：当请求中包含函数定义或从知识库检索到的大量上下文时，`auto` 模型可以自动切换到长上下文模型以容纳这些额外信息。
- **多模态输入判断**：开发者可以自己编写逻辑，判断用户输入是否包含图片。如果包含，则调用 `step-1v-auto`；如果不包含，则调用 `step-1-auto`，实现更灵活的多模态处理。

## 5. 注意事项

1.  **Token 计算误差**：由于 Token 的计算可能存在微小误差，平台在选择模型时可能会留有一定的缓冲空间。这可能导致在某些边缘情况下，选择的模型比预期中支持更长上下文的模型（例如，总 Token 为 8190 时，可能也会选择 32k 模型以确保成功率）。

2.  **官方工具的上下文处理**：当使用官方提供的工具（如 `web_search` 或知识库）时，这些工具添加的额外上下文长度会被自动计算在内，平台会智能选择足够大的模型来处理，开发者无需手动干预。